# Save Images

This is a demo of saving the [Extreme Heat Forecast](https://data.spatiafi.com/dataset/35) dataset.  We start by loading our App Credentials, and then using the [crop](https://docs.spatiafi.com/api/#operation/geojson_crop_api_image_crop_post) endpoint to fetch a part of the dataset cropped to an area of interest.

---

Install `authlib` (if not already installed) and importing some supporting libraries:

In [ ]:
!pip install requests authlib

In [ ]:
import errno
import json
import os
import time
from io import BytesIO

import matplotlib.pyplot as plt
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import ClientSecretJWT
from PIL import Image
from platformdirs import user_config_dir

### Load SpatiaFi App Credentials from file and create Authenticated `session`

We load previously generated App Credentials.

**If you have not generated App Credentials, please do so first by running the [App Authentication](https://docs.spatiafi.com/tutorials/app-authentication/) notebook**

In [ ]:
def load_spatiafi_credentials():
    if not os.environ.get("CLIENT_ID", False) or not os.environ.get(
        "CLIENT_SECRET", False
    ):
        file_location = os.path.join(
            user_config_dir("spatiafi", ensure_exists=True), "app_credentials"
        )

        if not os.path.isfile(file_location):
            print(
                f"File {file_location} not found. Have you generated and saved App Credentials?"
            )
            raise FileNotFoundError(
                errno.ENOENT, os.strerror(errno.ENOENT), file_location
            )

        with open(file_location) as f:
            data = json.load(f)

        for key, value in data.items():
            os.environ[key.upper()] = str(value)


load_spatiafi_credentials()

In [ ]:
# This function returns an `authlib` session, which is very similar to a `requests` session.
# It can be used to access the SpatiaFi API and will automatically refresh access tokens if they expire.


def get_session():
    client = OAuth2Session(
        os.environ["CLIENT_ID"],
        os.environ["CLIENT_SECRET"],
        token_endpoint="https://auth.spatiafi.com/api/v1/auth/jwt/token",
        grant_type="client_credentials",
        token_endpoint_auth_method=ClientSecretJWT(
            "https://auth.spatiafi.com/api/v1/auth/jwt/token"
        ),
    )
    client.fetch_token()
    return client


session = get_session()
print("Getting an access_token with client_id and client_secret")
response_data = session.fetch_token()
minutes, seconds = divmod(response_data["expires_at"] - int(time.time()), 60)


print(f"Access token expires in: {minutes} minutes, {seconds} seconds")

## Crop Dataset to AOI

Now that we have authenticated to the SpatiaFi API, we can use the `/api/image/crop` endpoint to download a specific Area of Interest (AOI).

In this case, the GeoJSON payload data was found by manually selecting an area using [geojson.io](geojson.io).  We could also generate these AOIs or read them from an existing file.

In [ ]:
url = "https://api.spatiafi.com/api/image/crop"

params = {
    "format": "png",
    "item_id": "ce-extreme-heat-risk-current-global-v1.0",
    "bidx": "1",
    "rescale": "0,4",  # this dataset has a range of values between 0-4, so we rescale the gray values to better see the image.
}

# North Saudi Arabia
payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.76491069149293, 32.29986592075784],
                [31.76491069149293, 23.092931581000542],
                [51.58140483659537, 23.092931581000542],
                [51.58140483659537, 32.29986592075784],
                [31.76491069149293, 32.29986592075784],
            ]
        ],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)

In [ ]:
response

### Open and save the generated image

Using [PIL](https://pillow.readthedocs.io/en/stable/) and matplotlib, we can quickly view the image in Jupyter before saving it to disk.

In [ ]:
# Open the image file from the response content
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.show()

## Save the Image to disk

In [ ]:
# Save image to disk
with open("output.png", "wb") as f:
    for chunk in response.iter_content(1024):
        f.write(chunk)